In [9]:
import os
import json
import pandas as pd

# 예산 데이터 폴더 경로
budget_dir = r"C:\Users\USER\OneDrive\Desktop\yonsei\3_2\ybigta\교육데이터\Kim-Yiji Contest_edudata yiji Database-schoolinfo_public_filtered"

# 파일명 리스트
budget_files = {
    2022: "경기_공립_고등_결산_세입_2022.json",
    2023: "경기_공립_고등_결산_세입_2023.json",
    2024: "경기_공립_고등_결산_세입_2024.json"
}

In [12]:
# 예산 데이터 저장용 리스트
budget_data = []

# 파일 읽고 데이터 쌓기
for year, file_name in budget_files.items():
    file_path = os.path.join(budget_dir, file_name)
    with open(file_path, encoding='utf-8') as f:
        data = json.load(f)
        for entry in data["list"]:
            budget_data.append({
                "학교명": entry["SCHUL_NM"],
                "도교육청": entry["ATPT_OFCDC_ORG_NM"],
                "연도": year,
                "예산_AMT1": entry["AMT1"]
            })

# DataFrame으로 변환
budget_df = pd.DataFrame(budget_data)

# 도별, 연도별 예산 합산 후 피벗벗
budget_lagged = (
    budget_df
    .groupby(["도교육청", "연도"])["예산_AMT1"]
    .sum()
    .reset_index()
    .rename(columns={"연도": "기준연도"})
)

print(budget_lagged)

     도교육청  기준연도       예산_AMT1
0  경기도교육청  2022  808335087430
1  경기도교육청  2023  855171127610
2  경기도교육청  2024  931104917780


In [14]:
# 보건실이용현황 데이터 로딩 및 도 단위 스코어 계산
# 스코어 계산 하.. 어떻게 하지..
health_path = r"C:\Users\USER\OneDrive\Desktop\yonsei\3_2\ybigta\교육데이터\edss_2차요청_데이터\학교정보공시\학생현황\0306. 보건실이용현황(21-23)(경기)(70%).csv"
health_df = pd.read_csv(health_path, encoding="utf-8")

health_lagged = (
    health_df
    .groupby(["시도교육청명", "공시년도"])["연간1인당보건실이용건수"]
    .mean()
    .reset_index()
    .rename(columns={"시도교육청명": "도교육청", "공시년도": "지표연도", "연간1인당보건실이용건수": "보건_스코어"})
)

# 시차 반영영
health_lagged["기준연도"] = health_lagged["지표연도"] + 1

merged_lagged = pd.merge(
    budget_lagged,
    health_lagged[["도교육청", "기준연도", "보건_스코어"]],
    on=["도교육청", "기준연도"],
    how="inner"
)

# 경기도교육청만 분석
gyeonggi_lagged = merged_lagged[merged_lagged["도교육청"] == "경기도교육청"].sort_values("기준연도")

print(gyeonggi_lagged)

     도교육청  기준연도       예산_AMT1    보건_스코어
0  경기도교육청  2022  808335087430  2.598988
1  경기도교육청  2023  855171127610  6.084377
2  경기도교육청  2024  931104917780  9.036657


In [ ]:
# 일단 단일로 분석해보자?

from sklearn.linear_model import LinearRegression

X = gyeonggi_lagged[["보건_스코어"]]
y = gyeonggi_lagged["예산_AMT1"]

model = LinearRegression()
model.fit(X, y)

# 예측 및 결과 계산
gyeonggi_lagged["예상_예산"] = model.predict(X)
gyeonggi_lagged["잔차"] = gyeonggi_lagged["예산_AMT1"] - gyeonggi_lagged["예상_예산"]
gyeonggi_lagged["편차_비율(%)"] = (gyeonggi_lagged["잔차"] / gyeonggi_lagged["예상_예산"] * 100).round(2)

print(gyeonggi_lagged)

     도교육청  기준연도       예산_AMT1    보건_스코어         예상_예산            잔차  편차_비율(%)
0  경기도교육청  2022  808335087430  2.598988  8.023466e+11  5.988462e+09      0.75
1  경기도교육청  2023  855171127610  6.084377  8.682294e+11 -1.305829e+10     -1.50
2  경기도교육청  2024  931104917780  9.036657  9.240351e+11  7.069830e+09      0.77


In [16]:
# 급식 파트 급식비 집행률로 보려고 했는데...
# 얘들 집행 잔액 없이 다 썼어.. 당연한거긴한데..
xlsx_path = r"C:\Users\USER\OneDrive\Desktop\yonsei\3_2\ybigta\교육데이터\edss_2차요청_데이터\학교정보공시\학교현황\0206. 급식실시현황(21-23)(경기)(70%).xlsx"
xls = pd.ExcelFile(xlsx_path)
meal_status_df = xls.parse("Sheet1")

meal_status_df["조리총원"] = (
    meal_status_df["조리보조원수"].fillna(0) +
    meal_status_df["조리사수"].fillna(0) +
    meal_status_df["영양사수"].fillna(0) + 1  # 0 나눗셈 방지
)

meal_status_df["급식서비스강도"] = meal_status_df["급식학생수"] / meal_status_df["조리총원"]

In [17]:
meal_quality_df = (
    meal_status_df
    .groupby(["시도교육청명", "공시년도"])["급식서비스강도"]
    .mean()
    .reset_index()
    .rename(columns={"급식서비스강도": "급식서비스_스코어"})
)

In [18]:
meal_quality_df["기준연도"] = meal_quality_df["공시년도"] + 1
meal_quality_df["급식_스코어"] = 1 / meal_quality_df["급식서비스_스코어"]

final_meal_score = (
    meal_quality_df[meal_quality_df["시도교육청명"] == "경기도교육청"]
    [["기준연도", "급식_스코어"]]
    .copy()
)
final_meal_score["도교육청"] = "경기도교육청"
final_meal_score.reset_index(drop=True, inplace=True)

print(final_meal_score)

   기준연도    급식_스코어    도교육청
0  2022  0.012834  경기도교육청
1  2023  0.012927  경기도교육청
2  2024  0.013331  경기도교육청
